In [1]:
import numpy as np
import wget 


In [2]:
from keras.layers import Dense

X = np.zeros(shape=(1, 200))
layer = Dense(units=50)
z = layer(X)

np.array(layer.bias).shape

2023-02-21 17:31:47.908619: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-21 17:31:47.950246: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-21 17:31:47.950541: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-21 17:31:47.951472: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

(50,)

In [3]:
layer.weights[0].shape, layer.weights[1].shape

(TensorShape([200, 50]), TensorShape([50]))

In [4]:
X = np.zeros(shape=(200))
W = np.ones(shape=(200, 50))
b = np.zeros(shape=(50))

T = W + b

Z = np.dot(X, W) + b

T.shape, Z.shape

((200, 50), (50,))

### Layer Perceptron
- init all Layer shape with list
- Create forward Prop
- Create Backward Prop

In [5]:
import tensorflow as tf
import numpy as np
from keras.initializers import GlorotUniform
import matplotlib.pyplot as plt
import math

def initialize_weight(input_shape : int, output_shape: int):
        """Initialize Layer Weight with Xavier Initialization

        Args:
            input_shape (int): dimension of input
            output_shape (int): dimension of output

        Returns:
            W : Numpy Array Weight of Layer
        """
        limit_value = np.sqrt(6 / (input_shape + output_shape))

        min_value = -limit_value
        max_value = limit_value
        
        W = np.random.uniform(low =min_value,
                              high=max_value,
                              size=(input_shape, output_shape))
        W = W.astype(np.float32)

        return (W)
    
def initialize_bias(weight_output_shape : int):
    """Initialize the Bias Vector with Zeros

    Args:
        weight_output_shape  (int): output dimension of Weight Matrix

    Returns:
        b : Numpy Array of Zeros
    """
    b = np.zeros(shape=(weight_output_shape))

    return (b)

In [77]:
from tensorflow.nn import softmax

class Sigmoid:
    
    def __init__(self):
        pass
    def __call__(self, x):
        return (1 / (1 + tf.exp(- x)))

class ReLU:
    def __init__(self):
        pass
    def __call__(self, x):
        return (tf.maximum(0, x))

class SoftMax:
    def __init__(self):
        pass
    def __call__(self, x):
        return (softmax(x))

class Linear:
    def __init__(self):
        pass
    def __call__(self, x):
        return (x)
    
def choose_activation(name : str):

    if (name == "sigmoid"):
        Activation = Sigmoid()

    elif (name == "relu"):
        Activation = ReLU()

    elif (name == "softmax"):
        Activation = SoftMax()

    else:
        Activation = Linear()
    
    return (Activation)

In [78]:
class Layer:
    
        def __init__(self, input_shape : int, units :int, activation : str):
            
            self.input_shape = input_shape
            self.output_shape = units
            self.W = initialize_weight(input_shape, units)
            self.b = initialize_bias(weight_output_shape=self.W.shape[1])
            self.activation = choose_activation(activation)
            self.A = None
            self.Z = None
        
        def __call__(self, X):
            self.Z = np.dot(X, self.W) + self.b
            self.A = self.activation(self.Z)
            return (self.A)

In [79]:
network_config = [
    Layer(input_shape=200, units=50, activation="relu"),
    Layer(input_shape=50, units=100, activation="relu"),
    Layer(input_shape=100, units=150, activation="relu"),
    Layer(input_shape=150, units=10, activation="softmax"),
]

for layer in network_config:
    print(f"Input Shape : {layer.input_shape} Output Shape : {layer.output_shape}")

Input Shape : 200 Output Shape : 50
Input Shape : 50 Output Shape : 100
Input Shape : 100 Output Shape : 150
Input Shape : 150 Output Shape : 10


In [80]:
for layer in network_config:
    print(layer)

In [122]:
from tabulate import tabulate

class MultiLayerPerceptron:

        def __init__(self, input_shape: int, network_config : list):

                self.input_shape = input_shape
                self.network = network_config
        
        def summary(self):
                print("Model")
                print("===============================================")
                
                total_params = 0
                table = list()
                headers=['Name', 'Output Shape', 'Number Params']

                input_layer = ["Input Layer", f"(None, {self.input_shape})", "0"]
                table.append(input_layer)
                
                for index, layer in enumerate(self.network):
                        
                        if index == len(self.network) - 1:
                                name = "Output Layer"
                        else :
                                name = f"layer {index + 1}"

                        output_shape = f"(None, {layer.output_shape})"
                        nbr_params = (layer.W.shape[0] * layer.W.shape[1])

                        element = [name, output_shape, f"{nbr_params:,}"]
                        
                        total_params += nbr_params
                        
                        table.append(element)
                        
                print(tabulate(table, headers))
                print("===============================================")
                print(f"Total Params : {total_params:,}")
                
        def forward(self, input_data):
                
                outputs = list()
                for index, layer in enumerate(self.network):
                        
                        if index == 0:
                                X = input_data
                        else :
                                last_layer = self.network[index - 1]
                                last_activation = last_layer.A
                                X = last_activation
                        
                        output = layer(X)
                
                return (output)
                        
        


```python
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
=================================================================
 input_1 (InputLayer)        [(None, 30, 2)]           0         
                                                                 
 flatten (Flatten)           (None, 60)                0         
                                                                 
 dense (Dense)               (None, 128)               7808      
                                                                 
 dense_1 (Dense)             (None, 7)                 903       
                                                                 
 reshape (Reshape)           (None, 7, 1)              0         
                                                                 
=================================================================
Total params: 8,711
Trainable params: 8,711
Non-trainable params: 0
_________________________________________________________________
```


In [123]:
X = np.zeros(shape=(1, 200))
model = MultiLayerPerceptron(input_shape=X.shape[1],
                             network_config=network_config)

In [124]:
model.summary()

Model
Name          Output Shape    Number Params
------------  --------------  ---------------
Input Layer   (None, 200)     0
layer 1       (None, 50)      10,000
layer 2       (None, 100)     5,000
layer 3       (None, 150)     15,000
Output Layer  (None, 10)      1,500
Total Params : 31,500


In [106]:
activations = model.forward(X)

for a in activations:
        print(a.shape)

(10,)
